# 08 - Encoder-Decoder (Seq2Seq) com Estabilização e Otimização

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from sklearn.metrics import mean_absolute_error

#https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/43795s

In [ ]:
data = pd.read_csv('ts_hr_feb_2020.csv.zip', parse_dates=['date'])

data['hour'] = data['date'].dt.hour

last_sunday = pd.to_datetime("2020-02-23")

train = data[data['date'] < last_sunday]
val = data[data['date'] >= last_sunday]

In [ ]:
def prep_seqs(df, l=24, h=24):
    
    X = []
    Y = []
    for i in range(l,df.shape[0]-h):
        f = df.iloc[i-l:i]['sales_value'] #hoje
        y = df.iloc[i:i+h]['sales_value'] # 1 dia depois

        X.append(f.values)
        Y.append(y.values)

    X = np.array(X)
    Y = np.array(Y)

    X = np.expand_dims(X, -1)
    X = np.swapaxes(X, 0,1)
    
    return torch.from_numpy(X).float(), torch.from_numpy(Y).float()

In [ ]:
X, Y = prep_seqs(train)
Xt, Yt = prep_seqs(val)

mean_train = X.mean(1, keepdims=True)
std_train = X.std(1,  unbiased=False, keepdims=True)

meanY_train = Y.mean(0, keepdims=True)
stdY_train = Y.std(0, unbiased=False, keepdims=True)

X -= mean_train
X /= std_train

Xt -= mean_train
Xt /= std_train

Y -= meanY_train
Y /= stdY_train

X = X.cuda()
Y = Y.cuda()
Xt = Xt.cuda()

# cuda = torch.device('cuda') 
#Y.to(cuda)

In [ ]:
class EncoderDecoderRNN(nn.Module):
    
    def __init__(self, hidden_size):
        super(EncoderDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.encoder = nn.GRU(1, self.hidden_size)
        self.decoder = nn.GRU(1, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, 1)
        
    def forward(self, input):
        
        output_encoder, _ = self.encoder(input)
        hidden_encoder = output_encoder[-1:, :, :].cuda()
        
        last_known_value = input[-1:,:,:].cuda()
        #print(last_known_value.shape)
        last_hidden = hidden_encoder
        #print(last_hidden.shape)
        
        outs = []
        for step in range(24):
            output_decoder, _ = self.decoder(last_known_value, last_hidden)
            
            last_hidden = output_decoder[-1:, :, :]
            p = self.out(last_hidden)
            
            last_known_value = p
            outs.append(p)
            
        #print(outs[0].shape, outs[1].shape)
            
        outs = torch.cat(outs).squeeze()
        outs = torch.transpose(outs, 0, 1)
        
        return outs

In [ ]:
def run_seq2seq(params):
    
    hidden_size, lr = params
    hidden_size = int(hidden_size)
    
    total_p = np.zeros((Yt.shape[0], Yt.shape[1], 10))

    for seed in range(10):
        torch.manual_seed(seed)
        np.random.seed(seed)

        edrnn = EncoderDecoderRNN(hidden_size).cuda()
        criterion = nn.L1Loss()
        lstm_optimizer = optim.Adam(edrnn.parameters(), lr=lr)

        for i in range(100):
            edrnn.zero_grad()

            o = edrnn(X)
            loss = criterion(o, Y)
            loss.backward()
            lstm_optimizer.step()
            #print(loss.item())

            p = edrnn(Xt)
            p = p.detach().cpu()

            p *= stdY_train
            p += meanY_train
            p = p.numpy().squeeze()

        #print("Seed = {} - Erro: {}".format(seed, mean_absolute_error(Yt.numpy(), p)))
        total_p[:, :, seed] = p
    
    e = mean_absolute_error(Yt.numpy(), total_p.mean(axis=-1))
    
    print("\nhidden_size = {}, lr = {}".format(*params))
    print("Ensemble Avg Score = {}\n".format(e))
    
    return e

In [ ]:
from skopt import gp_minimize

space = [(10, 300),
         (1e-4, 1e-1, 'log-uniform')]

res = gp_minimize(run_seq2seq, space, random_state=4, verbose=1, n_calls=11)

In [ ]:
# 342 - erro sem tuning LSTM
# 324.39008277809006 - menor erro LSTM tunada

In [ ]:
res.x

# Fim